Imports

In [10]:
import tensorflow as tf
import tensorflow_datasets as tfds
!pip install tensorflow_text
import tensorflow_text as tf_text
from google.colab import drive
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import BatchNormalization, Reshape, Conv2DTranspose, Flatten, Conv2D, Dense, Dropout

     |████████████████████████████████| 4.9 MB 5.0 MB/s 


In [2]:
# bash code to mount the drive
drive.mount('/content/drive')
os.chdir('drive/MyDrive')

Mounted at /content/drive


# Preparing the data

In [11]:
def clean_text(corpus):
    """ removes nex line and special characters from a given text using regex
        Args:
            corpus (str): raw text
    """
    # replace all whitespace characters with a space (to remove tab and newline, etc) but still be able to split by space
    corpus = re.sub('\s', ' ', corpus)
    # remove all special characters (everything that is not a letter from a-z or whitespace)
    corpus = re.sub(r'[^a-zA-z\s]', '', corpus)
    
    return corpus

In [15]:
# convert to lower case
corpus =  clean_text(open('/content/drive/MyDrive/bible.txt').read().lower())

# Tokenize the string into word-tokens by using one of the word-level tokenizers from tensorflow-text
tokenizer = tf_text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(corpus)

# get the 10000 most frequent tokens
# count the input by passing through a regex and lowercase it
top_tokens = []
# do it with regex or not? (re.findall(r"[a-z']+", corpus))
c = Counter(corpus)
# return the `most common` 10000 items
top_tokens = [w for w,_ in c.most_common(10000)]

In [ ]:
# Create Dataset with target context pairs [target_word, context_word] with subsampling
context_window = 4

# enumerate over top 10000 tokens
for index, token in enumerate(top_tokens):
    # loop over conext window size
    for context_index in range(1, context_window + 1):
        if (index - context_index) > 0 and (index + context_index) < total_word_num:
                context_left = top_tokens[index - context_index]
                context_right = top_tokens[index + context_index]

# Skipgram Model

In [ ]:
class Skipgram(tf.keras.Model):
    
    def __init__(self):
        
    def build(self):

    def call(self):
        